In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17937641931004275119
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16201483151841789243
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 17017570189779161252
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14648777152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17395291128015835598
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [0]:
# import pandas as pd
# import numpy as np
# import tensorflow as tf

# import os.path
# import pickle
# from keras.preprocessing.text import Tokenizer, one_hot
# from keras.preprocessing.sequence import pad_sequences
# from keras.models import Sequential

# from keras.layers.core import Activation, Dropout, Dense
# from keras.layers import Flatten, LSTM
# from keras.layers import GlobalMaxPooling1D
# from keras.models import Model
# from keras.layers.normalization import BatchNormalization

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import precision_score, recall_score, f1_score
# from keras.utils.np_utils import to_categorical
# from keras.callbacks import EarlyStopping
# from keras.layers import Dropout, Input, Embedding
# from keras.layers.merge import Concatenate

# import matplotlib.pyplot as plt
# import nltk
# import re
# import seaborn as sns

# from tqdm import tqdm
# from sklearn.model_selection import train_test_split
# from nltk.tokenize import word_tokenize
# from nltk import pos_tag
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# import nltk

# from numpy import array
# from numpy import asarray
# from numpy import zeros

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter
import nltk
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

In [0]:
real = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/2020_04_10000_real_tweets.csv', header = None)
fake = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/2020_04_10000_fake_tweets_conditional.csv', header = None)


In [0]:
real.columns = ['tweet']

In [0]:
real['index'] = real.index

In [0]:
fake.columns = ['tweet']
fake['index'] = fake.index

In [0]:
cols = real.columns.tolist()
cols = cols[-1:] + cols[:-1]

In [0]:
real = real[cols]

In [0]:
cols = fake.columns.tolist()
cols = cols[-1:] + cols[:-1]
fake = fake[cols]

In [11]:
real['label'] = 1
fake['label'] = 0
real.head()

,index,tweet,label
0,0,#Taiwan isn't a member of the #WHO. Taiwan tr...,1
1,1,From the DJs to the CJs.. Look whose in the PJ...,1
2,2,The @ClinicalBioinfo led by @xdopazo releases ...,1
3,3,IMF expects #Greece to be the worst hit econom...,1
4,4,More on #COVID19 - China witnessed a slight in...,1


In [12]:
fake.head()

,index,tweet,label
0,0,"#Taiwan isn't a member of the US, but it has s...",0
1,1,From the DJs to the new single:,0
2,2,The @ClinicalBioinfo led by @xdopazo seems to ...,0
3,3,IMF expects #Greece to be one of the European ...,0
4,4,More on #COVID19 - China’s COVID-19 death toll...,0


In [13]:
fake['tweet'][0]

"#Taiwan isn't a member of the US, but it has some nice trade ties, and China can keep them in the same league as the rest of the world."

In [14]:
combined = pd.concat([fake, real])
features = combined['tweet']
labels = combined['label']
combined = combined.sample(frac = 1).reset_index(drop = True)
combined['index'] = combined.index
combined.tail()

,index,tweet,label
19995,19995,Woman Pled with Amazon Assistant to Task COVID-19,0
19996,19996,Here’s a quick sum up on how #COVID19 has affe...,0
19997,19997,"Good afternoon friends, a perfect sunrise afte...",0
19998,19998,Damn. China got you sitting at home?,0
19999,19999,Bell adds capacity to networks and gives hope ...,0


In [0]:
combined['tweet'] = combined['tweet'].astype(str).str.replace('[^a-zA-Z]',' ').str.lower()

In [0]:
X_body_text = combined['tweet'].values
y = combined['label'].values

In [17]:
X_body_text

array(['doctors warn of deadly coronavirus ', 'lockdown uniform   ',
       'some fanart for the charity stream  come and join jack       jack septic eye  hopefromhome  stayhome  worldhealthday',
       ...,
       'good afternoon friends  a perfect sunrise after  coronavirus',
       'damn  china got you sitting at home ',
       'bell adds capacity to networks and gives hope to hospitals in covid    crisis  '],
      dtype=object)

In [18]:
y

array([0, 1, 1, ..., 0, 0, 0])

In [0]:
from sklearn.naive_bayes import MultinomialNB

nb_body = MultinomialNB()

In [0]:
# def mod(X_body_text, nb_body):
#     upperLimitRange = range(2,20)
#     max_df_range = np.linspace(0.2,0.9,71)
#     max_f1_train = 0
#     max_accuracy_train = 0
#     max_f1_acc_train = 0
#     max_f1_test = 0
#     max_accuracy_test = 0
#     max_f1_acc_test = 0
#     best_ngram = 0
#     best_max_df = 0
#     count = 0
#     for i in upperLimitRange:
#         for j in max_df_range:
#             tfidf = TfidfVectorizer(ngram_range=(1,i), max_df = j, min_df = 0.01)
#             X_body_tfidf = tfidf.fit_transform(X_body_text)
#             indices = combined.index.values
#             X_body_tfidf_train, X_body_tfidf_test, y_body_train, y_body_test, indices_body_train, indices_body_test = train_test_split(X_body_tfidf, y, indices, test_size = 0.2, random_state=42)
#             nb_body.fit(X_body_tfidf_train, y_body_train)
#             y_body_train_pred = nb_body.predict(X_body_tfidf_train)

#             f1_score_train = f1_score(y_body_train, y_body_train_pred, average='macro')
#             accuracy_score_train = accuracy_score(y_body_train, y_body_train_pred)

#             if f1_score_train > max_f1_train:
#                 max_f1_train = f1_score_train
#                 max_f1_acc_train = accuracy_score_train
#                 best_ngram = i
#                 best_max_df = j

#             if accuracy_score_train > max_accuracy_train:
#                 max_accuracy_train = accuracy_score_train
#                 print(max_accuracy_train)
            
#             count = count + 1
#             print(count)

#             # y_body_pred = nb_body.predict(X_body_tfidf_test)
            
#             # f1_score_test = f1_score(y_body_test, y_body_pred, average='macro')
#             # accuracy_score_test = accuracy_score(y_body_test, y_body_pred)

#             # if f1_score_test > max_f1_test:
#             #     max_f1_test = f1_score_test
#             #     max_f1_acc_test = accuracy_score_test
#             #     best_ngram = i
#             #     best_max-df = j
            
#             # if accuracy_score_test > max_accuracy_test:
#             #     max_accuracy_test = accuracy_score_test
#             #     print(max_accuracy_test)

#     return best_ngram, best_max_df, max_f1_train, max_f1_acc_train


# best_ngram, best_max_df, max_f1_train, max_f1_acc_train = mod(X_body_text, nb_body)
# print("Best ngram value = ", best_ngram)
# print("Best max_df_value = ", best_max_df)
# print("Best training f1_score = ", max_f1_train)
# print("Accuracy for the best f1_score = ", max_f1_acc_train)





In [0]:
best_ngram = 2
best_max_df = 0.62


In [0]:
tfidf = TfidfVectorizer(ngram_range=(1,best_ngram), max_df= best_max_df, min_df= 0.01)

In [0]:
X_body_tfidf = tfidf.fit_transform(X_body_text)

In [0]:
indices = combined.index.values

In [0]:
X_body_tfidf_train, X_body_tfidf_test, y_body_train, y_body_test, indices_body_train, indices_body_test = train_test_split(X_body_tfidf, y, indices, test_size = 0.2, random_state=42)

In [26]:
combined.loc[indices_body_train].groupby('label').agg('count')

,index,tweet
label,,
0,7971,7971
1,8029,8029


In [27]:
combined.loc[indices_body_test].groupby('label').agg('count')

,index,tweet
label,,
0,2029,2029
1,1971,1971


In [0]:
from sklearn.naive_bayes import MultinomialNB

In [0]:
nb_body = MultinomialNB()

In [30]:
nb_body.fit(X_body_tfidf_train, y_body_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [0]:
y_body_train_pred = nb_body.predict(X_body_tfidf_train)

In [32]:
print('Naive Bayes Train - F1 and Accuracy Scores:')
print('F1 score {:.4}%'.format(f1_score(y_body_train, y_body_train_pred, average='macro')*100 ))
print('Accuracy score {:.4}%'.format(accuracy_score(y_body_train, y_body_train_pred)*100))

Naive Bayes Train - F1 and Accuracy Scores:
F1 score 65.26%
Accuracy score 65.36%


In [33]:
np.where(y_body_train != y_body_train_pred)

(array([    1,     2,     4, ..., 15995, 15996, 15999]),)

In [34]:
combined.loc[[52, 449]]

,index,tweet,label
52,52,fda shifts its stance on covid response to ...,0
449,449,they havent posted in bout days i am so h...,0


In [0]:
y_body_pred = nb_body.predict(X_body_tfidf_test)

In [36]:
print('Naive Bayes Test - F1 and Accuracy Scores: (Conditional)')
print('F1 score {:.4}%'.format(f1_score(y_body_test, y_body_pred, average='macro')*100 ))
print('Accuracy score {:.4}%'.format(accuracy_score(y_body_test, y_body_pred)*100))

Naive Bayes Test - F1 and Accuracy Scores: (Conditional)
F1 score 63.26%
Accuracy score 63.32%


In [37]:
np.where(y_body_test != y_body_pred)

(array([   3,    6,    7, ..., 3994, 3996, 3997]),)